In [17]:
import numpy as np # LA tools
import pandas as pd # Data processing tools
import matplotlib.pyplot as plt # Visualization tools
from scipy.sparse import csr_matrix # Effective data storage structure

import implicit # Effective RecSys models realization
import pickle
import time
from datetime import datetime
import yaml

from PIL import Image
import requests
from io import BytesIO

from Recommender import RecSys
from Processors import InputProcessor, OutputProcessor
from User import User, UserError
from DataStore import DataStore
from TitleModeler import TitleModeler
from StyleRecommender import StyleRecommender

# RecSys:

In [18]:
working_data_path = "./Data/Working_Data/second_model/"

with open(working_data_path + "model_implicit_als.pickle", "rb") as f:
    model = pickle.load(f)
    
with open(working_data_path + "data_store.pickle", "rb") as f:
    dt_store = pickle.load(f)

with open(working_data_path + "model_style.pickle", "rb") as f:
    stylerec = pickle.load(f)

In [19]:
with open('questionnaire.yaml') as f:
    survey_dict = yaml.safe_load(f)

In [20]:
book_features = pd.read_csv("Data/u_process_text/good_overall_stat.csv", index_col="Unnamed: 0")
book_info = pd.read_csv("Data/u_process_text/good_text_info.csv", index_col="Unnamed: 0")

candidates = book_features.index[:9].values
cand_to_ind = {candidates[i]: i for i in range(len(candidates))}

ind_to_text = dict()
for i, name in enumerate(candidates):
   with open("Data/u_process_text/equal_txt/" + name, "r") as f:
      tx = f.read()
      ind_to_text[i] = ".".join(tx.split(".")[500:501])

db = pd.DataFrame(columns=["name", "surname", "chat_id", "style", "preference", "style_choice", "preference_choice"])

# BOT:

In [21]:
import telegram
from googlesearch import search

from telegram import Update

from telegram.ext import (
    Updater,
    CommandHandler,
    PollAnswerHandler,
    MessageHandler,
    Filters,
    CallbackContext,
)


import logging

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO,
)

In [22]:
def start(update: Update, context: CallbackContext) -> None:
    """Inform user about what this bot can do"""
    
    print("/start called...")

    message = context.bot.send_message(
        chat_id=update.effective_chat.id,
        text='Please select:\n\n /start to get this message;' 
             + '\n\n/preference to get recommendations;'
             + '\n\n/help to get information about the Bot;'
    )

    payload = {
        update.effective_chat.id:{
            "message_id": message.message_id,
            "chat_id": update.effective_chat.id,
            "count": 1,
            "answers": ["", "", "", ""],
        }
    }
    context.bot_data.update(payload)


def help_me(update: Update, context: CallbackContext) -> None:
    """Inform user about Bot"""
    
    print("/help called...")
    text = (
        "***** *****  Book Spark  2.0  ***** *****\n\n" + "Innovative recommender system based on stylometrics."
        + "\n\nIf you would like to get recommendations you have 2 mixed recommendations:"
        + "\n1. Get new titles that you have not seen yet based on author styles - using polls;"
        + "\n2. Get titles that might be relevant based on your past preference;"
        + "\n\n We will add more possibilities in the next versions:)"
    )
    update.message.reply_text(text)

In [23]:
def preferences(update: Update, context: CallbackContext) -> None:
    """ Start collecting preferences """
    
    print("/preference called...")
    message = context.bot.send_message(
        chat_id=update.effective_chat.id,
        text=context.bot_data["qa"]["questions"][0]
    )
    
    payload = {
        update.effective_chat.id:{
            "message_id": message.message_id,
            "chat_id": update.effective_chat.id,
            "count": 1,
            "answers": ["", "", "", ""],
        },
        "check":1
    }
    context.bot_data.update(payload)
    

def receive_answer(update: Update, context: CallbackContext) -> None:
    """ Get answer and call poll"""
    

    if context.bot_data["check"] == 0:
        context.bot.send_message(
            chat_id=update.effective_chat.id,
            text=":)"
        )
        return


    answer = update.message.text
    num_calls = context.bot_data[update.message["chat"]["id"]]["count"]

    
    if num_calls == 5:
        context.bot.send_message(
            chat_id=update.effective_chat.id,
            text=(
                    'Please select:\n\n /start to get this message;' 
                    + '\n\n/preference to get recommendations;'
                    + '\n\n/help to get information about the Bot;'
                )
        )
        return 
    

    # Save first title
    
    context.bot_data[update.message["chat"]["id"]]["answers"][num_calls - 1] = answer 
    
    chat_id = context.bot_data[update.message["chat"]["id"]]["chat_id"]
    first_name = update.message["chat"]["first_name"]
    last_name = update.message["chat"]["last_name"]

    result_dict = {
        "name": first_name,
        "surname": last_name,
        "chat_id": chat_id,
        "style": False,
        "preference": True,
        "style_choice": [],
        "preference_choice": context.bot_data[chat_id]["answers"]
    }
    
    db.loc[str(chat_id) + "_" + str(datetime.now())] = result_dict

    print(f"QA: {num_calls}; ChatID: {chat_id};"
          + f" Name: {first_name}; Surname: {last_name}; Answer_{num_calls-1}: {answer}\n")

    
    
    # Get next title
    
    plug = "w1a2s3t4e5"
    user = User()
    user.add_preference(key="book1", value=context.bot_data[chat_id]["answers"][0])
    user.add_preference(key="book2", value=plug)
    user.add_preference(key="book3", value=plug)

    recsys = RecSys(model, dt_store)
    titles, user_vec = recsys.recommend(user, 50)

    if user_vec.sum() > 1.0:
        print("Got random input...\n")
        context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="Title you have just put is invalid up to the system!"
        )

        context.bot.send_message(
            chat_id=update.effective_chat.id,
            text="Put one more:\n"
        )
        return 

    
    title_modeler = TitleModeler()
    ttls = title_modeler.get_titles_poll(titles, user_vec, dt_store.books_db, "authors", ",", 3)
    ttls = [ttl[0] for ttl in ttls]
    
    message = context.bot.send_poll(
        update.effective_chat.id,
        context.bot_data["qa"]["questions"][1],
        ttls,
        is_anonymous=False,
        allows_multiple_answers=False,
    )
    
    payload = {
            chat_id:{
                "message_id": message.message_id,
                "chat_id": update.effective_chat.id,
                "count": num_calls + 1,
                "answers": context.bot_data[chat_id]["answers"],
                "poll_questions": ttls,
            }
        }
    
    context.bot_data.update(payload)


def receive_poll_answer(update: Update, context: CallbackContext) -> None:
    """Summarize a users poll vote"""

    answer = update.poll_answer
    chat_id = answer["user"]["id"]

    num_calls = context.bot_data[chat_id]["count"]

    first_name = answer["user"]["first_name"]
    last_name = answer["user"]["last_name"]

    result_dict = {
        "name": first_name,
        "surname": last_name,
        "chat_id": chat_id,
        "style": False,
        "preference": True,
        "style_choice": [],
        "preference_choice": context.bot_data[chat_id]["answers"]
    }
    
    db.loc[str(chat_id) + "_" + str(datetime.now())] = result_dict

    print(f"QA: {num_calls}; ChatID: {chat_id};"
          + f" Name: {first_name}; Surname: {last_name}; Answer_{num_calls-1}: {answer}\n")

    print(context.bot_data[chat_id]["poll_questions"][answer["option_ids"][0]])


    context.bot_data[chat_id]["answers"][num_calls - 1] = context.bot_data[chat_id]["poll_questions"][answer["option_ids"][0]]

    if num_calls == 3:
        print("STYLE!!!!!!")
        title_info = stylerec.give_titles()
        ttls = [" ;".join(ttl[3]) for ttl in title_info]
    
        context.bot.send_message(
                chat_id=chat_id,
                text="*** Read 3 narratives different in style: ***\n",
            )

        for i, j  in enumerate(ttls):
            context.bot.send_message(
                chat_id=chat_id,
                text= f"{i + 1}. "+ j + "\n",
            )
        
        message = context.bot.send_poll(
            chat_id,
            context.bot_data["qa"]["questions"][num_calls],
            [f"{i}. \"{title_info[i-1][1]}\" - {title_info[i-1][2]}" for i in range(1, len(title_info) + 1)],
            is_anonymous=False,
            allows_multiple_answers=False,
        )

        payload = {
                chat_id:{
                    "message_id": message.message_id,
                    "chat_id": chat_id,
                    "count": num_calls + 1,
                    "answers": context.bot_data[chat_id]["answers"],
                    "poll_questions": [ttl[0] for ttl in title_info],
                }
            }
    
        context.bot_data.update(payload)
        return

    elif num_calls == 4: 
        
        user = User()
        user.add_preference(key="book1", value=context.bot_data[chat_id]["answers"][0])
        user.add_preference(key="book2", value=context.bot_data[chat_id]["answers"][1])
        user.add_preference(key="book3", value=context.bot_data[chat_id]["answers"][2])
        
        recsys = RecSys(model, dt_store)
        titles, user_vec = recsys.recommend(user, 50)
        
        title_modeler = TitleModeler()
        ttls = title_modeler.get_titles_poll(titles, user_vec, dt_store.books_db, "authors", ",", 5, True)

        context.bot.send_message(
            chat_id=chat_id,
            text="*** Preference Recommendations: ***\n"
        )
        
        for i, title in enumerate(ttls):
            
            context.bot.send_message(
                chat_id=chat_id,
                text=f"{i+1}. \"{title[0]}\"" + f" compatibility score: {title[1][0]:.2}", 
            )
            
            url = dt_store.books_db["image_url"].iloc[title[1][1]]

            context.bot.send_photo(
                chat_id=chat_id,
                photo=url, 
            )

        context.bot.send_message(
            chat_id=chat_id,
            text="*** Style Recommendations: ***\n",
        )

        idx = context.bot_data[chat_id]["answers"][3]
        title_info = stylerec.give_similar_titles(idx)
        
        for i, title in enumerate(title_info):
            
            context.bot.send_message(
                chat_id=chat_id,
                text=f"{i+1}. \"{title[0]}\" - {title[1]}" , 
            )
        

        context.bot.send_message(
            chat_id=chat_id,
            text="*** You got all the recommendations! ***"
        )

        context.bot.send_message(
            chat_id=chat_id,
            text="*** Would you please give us your feedback: ***\n"
        )

        message = context.bot.send_message(
            chat_id=chat_id,
            text="https://forms.gle/thzM6oyepW6pY4QZ9"
        )
        
        context.bot_data[chat_id]["answers"] = ["", "", "", ""]

        payload = {
            chat_id:{
                "message_id": message.message_id,
                "chat_id": chat_id,
                "count": 5,
                "answers": context.bot_data[chat_id]["answers"],
            }
        }
    
        context.bot_data.update(payload)
        
        print("The End")
        return
       
    
    elif num_calls > 4:
        context.bot.send_message(
            chat_id=chat_id,
            text="num_calls > 4"
        )
        return
    
    plug = "w1a2s3t4e5"
    user = User()
    user.add_preference(key="book1", value=context.bot_data[chat_id]["answers"][0])
    user.add_preference(key="book2", value=context.bot_data[chat_id]["answers"][1])
    user.add_preference(key="book3", value=plug)

    recsys = RecSys(model, dt_store)
    titles, user_vec = recsys.recommend(user, 50)
    
    title_modeler = TitleModeler()
    ttls = title_modeler.get_titles_poll(titles, user_vec, dt_store.books_db, "authors", ",", 3)
    ttls = [ttl[0] for ttl in ttls]

    message = context.bot.send_poll(
        chat_id,
        context.bot_data["qa"]["questions"][num_calls],
        ttls,
        is_anonymous=False,
        allows_multiple_answers=False,
    )

    payload = {
            chat_id:{
                "message_id": message.message_id,
                "chat_id": chat_id,
                "count": num_calls + 1,
                "answers": context.bot_data[chat_id]["answers"],
                "poll_questions": ttls,
            }
        }
    
    context.bot_data.update(payload)

In [ ]:
TOKEN = #PUT HERE YOUR TELEGRAM TOKEN

updater = Updater(token=TOKEN, use_context=True)
dispatcher = updater.dispatcher


dispatcher.add_handler(CommandHandler('start', start))
dispatcher.add_handler(CommandHandler('preference', preferences))
dispatcher.add_handler(CommandHandler('help', help_me))
dispatcher.add_handler(PollAnswerHandler(receive_poll_answer))
dispatcher.add_handler(MessageHandler(Filters.text & (~Filters.command), receive_answer))

questions_dict = {
        "qa": {
            "questions": [
                "Specify the book title you like:\n",
                "Choose one book you like:\n",
                "Choose another book:\n",
                "Select from 3 styles below:\n",
            ]
        },
        "check": 0,
    }
questions_dict["style"] = survey_dict  
dispatcher.bot_data.update(questions_dict)

In [ ]:
updater.start_polling()

updater.idle()

2022-01-13 12:35:45,963 - apscheduler.scheduler - INFO - Scheduler started
2022-01-13 12:36:33,121 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2022-01-13 12:36:33,121 - apscheduler.scheduler - INFO - Scheduler has been shut down
